In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

1. Import data
2. Select Test and train data
3. Divide data in target and features
4. Search and Select Relevant Features
5. Fit MultiTarget Classifier
6. Test accuracy and compare
7. Fit MultiTarget Regressor (Try to use only success cases after analysing)
8. Test accuracy and compare

--> Create ranks using ranking rules

9. Develope ranking method (ART Method)
10. Derive absoluate ranks to check accuracy
11. Predict ranks of test set and find accuracy (Do not calculate but predict)


Predict Rank -> Check Relevancy -> Use time to remove ties

### 1. Import data

In [2]:
data1900 = pd.read_csv("F:\\ML\\MultiTarget\\data1900.csv")

### 2. Select Test and train data

In [15]:
# Selecting data in test and train
Test  = data1900[data1900.Category=='Test']
Train = data1900[data1900.Category == 'Train']
print("Test Shape:", Test.shape,"\t train shape:", Train.shape)

('Test Shape:', (465, 157), '\t train shape:', (1500, 157))


### 3. Select target and features

In [109]:
# Features starts from 33rd columns
Test_features  = Test.iloc[:,36:]
Train_features = Train.iloc[:,36:]

# Classification targets (Reasoner to be predicted)
Test_Targets_c = Test[filter(lambda x: (x.endswith('CORR')) , Test.columns.tolist())]
Train_Targets_c = Train[filter(lambda x: (x.endswith('CORR')) , Train.columns.tolist())]

for t in Test_Targets_c.columns.tolist():
    Test_Targets_c.loc[Test_Targets_c[t] != 'Correct', t] = 0
    Test_Targets_c.loc[Test_Targets_c[t] == 'Correct', t] = 1
    
    Train_Targets_c.loc[Train_Targets_c[t] != 'Correct', t] = 0
    Train_Targets_c.loc[Train_Targets_c[t] == 'Correct', t] = 1

lbl = preprocessing.LabelEncoder()

for t in Test_features.columns.tolist():
    if(Train_features[t].dtype == 'O'):
        Train_features[t] = lbl.fit_transform(Train_features[t])
        Test_features[t] = lbl.fit_transform(Test_features[t])
'''
for t in Test_Targets_c.columns.tolist():
    if(Train_Targets_c[t].dtype == 'object'):
        Test_Targets_c[t] = lbl.fit_transform(Test_Targets_c[t])
        Train_Targets_c[t] = lbl.fit_transform(Train_Targets_c[t])
    
'''    

#Regressor targets (Execution time of each reasoner)
Test_Targets_r = Test[filter(lambda x: (x.endswith('ETime')) , Test.columns.tolist())]
Train_Targets_r = Train[filter(lambda x: (x.endswith('ETime')) , Train.columns.tolist())]


In [39]:
print(Test_features.shape,Train_Features.shape, Test_Targets_c.shape, Train_Targets_c.shape, Test_Targets_r.shape, Train_Targets_r.shape)

((465, 124), (1500, 124), (465, 10), (1500, 10), (465, 10), (1500, 10))


### 4. Select Relevenat features

### 5. Fit Multilable classifier

In [104]:
clf_rf = RandomForestClassifier(n_estimators=100,random_state=45)
mclf_rf = MultiOutputClassifier(clf_rf, n_jobs=-1)
mclf_rf.fit(Train_features, Train_Targets_c)

In [118]:
mclf_rf.score(Test_features, Test_Targets_c)

0.74838709677419357

In [123]:
m_relevance_vector = mclf_rf.predict(Test_features)

In [117]:
relevance_vector = pd.DataFrame(data = Train_Targets_c, columns = Train_Targets_c.columns.tolist())
for c in  Train_Targets_c.columns.tolist():
    clf_rf.fit(Train_features, Train_Targets_c[c])
    relevance_vector[c] =  clf_rf.predict(Train_features)
    #print(c,':',clf_rf.score(Test_features, Test_Targets_c[c]))
relevance_vector.head()

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
0,1,0,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,1,1,1,1
3,1,0,0,0,0,1,0,0,0,0
5,0,0,1,0,0,1,0,1,0,0
6,0,0,1,0,0,1,0,1,0,0


In [124]:
rgs_rf = RandomForestRegressor(n_estimators=100,random_state=45)
mrgs_rf = MultiOutputRegressor(rgs_rf, n_jobs=-1)


In [125]:
mrgs_rf.fit(Train_features, Train_Targets_r)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=45,
           verbose=0, warm_start=False),
           n_jobs=-1)

In [126]:
mrgs_rf.score(Test_features, Test_Targets_r)

0.70094398753695764

In [128]:
m_timerank_vector = mrgs_rf.predict(Test_features)

In [130]:
timerank_vector = pd.DataFrame(data = Train_Targets_r, columns = Train_Targets_r.columns.tolist())
for c in  Train_Targets_r.columns.tolist():
    rgs_rf.fit(Train_features, Train_Targets_r[c])
    timerank_vector[c] =  rgs_rf.predict(Train_features)
    #print(c,':',clf_rf.score(Test_features, Test_Targets_c[c]))
timerank_vector.head()

,Racer_ETime,ELepHant_ETime,TrOWL_ETime,JFact_ETime,MOReHermiT_ETime,Konclude_ETime,FaCT++_ETime,ELK_ETime,Pellet-OA4_ETime,HermiT-OA4_ETime
0,74953.771667,36.210000,5234.950000,135538.04500,2995.300000,3858.708333,17558.080000,1725.930000,8253.410000,14318.990000
1,4644.003333,13.050000,1500.630000,6633.27000,1649.195000,389.990000,2949.203667,859.973333,2405.470000,2495.950000
3,27288.220000,22.430000,4708.596667,40777.08000,47703.430000,1207.210000,39842.053333,456.650000,37450.360000,33430.180000
5,207627.850000,679.201417,5897.824417,204817.89725,203779.115583,4026.699500,65112.786250,2221.227417,209744.002417,177813.587167
6,207627.850000,679.201417,5897.824417,204817.89725,203779.115583,4026.699500,65112.786250,2221.227417,209744.002417,177813.587167


## Calculation of reasoner ranks using following rules
1. Arrange reasoners according to relevancy
    Correct > Unexpected > Timeout >Execution Error
2. Arrange each reasoner according to execution time

In [276]:
# Select data

Train_Targets_c2 = Train[filter(lambda x: (x.endswith('CORR')) , Train.columns.tolist())]
Test_Targets_c2 = Test[filter(lambda x: (x.endswith('CORR')) , Test.columns.tolist())]

#print(Train_Targets_c2.tail())

for t in Test_Targets_c.columns.tolist():
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Correct', t] = 1
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Unexpected', t] = 2
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Timeout', t] = 3
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Exec_err', t] = 4
    
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Correct', t] = 1
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Unexpected', t] = 2
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Timeout', t] = 3
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Exec_err', t] = 4
    
Train_Targets_c2.tail()

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
1959,1,4,2,2,1,1,1,2,1,1
1960,1,4,2,3,1,1,1,2,3,1
1961,1,4,2,3,1,1,1,2,3,3
1963,1,4,2,2,1,1,1,2,1,1
1964,1,4,1,3,1,1,1,1,1,1


In [283]:
reasoner_ranks_train = Train_Targets_c2.copy(deep=True)

for index, row in Train_Targets_c2.iterrows():
    one_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    two_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    three_df = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    four_df  = pd.DataFrame(columns = ['r_name','r_time','r_rank'])

    
    for c in Train_Targets_c2.columns:
        #print(Test_Targets_c2.loc[index][c])
        r = c.split('_')[0]
        #print(Test_Targets_c2.loc[index])
 #       print(c,Test_Targets_c2.loc[index][c])
        if(Train_Targets_c2.loc[index][c] == 1):
            one_l.append(c.split('_')[0])
            one_df = one_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Train_Targets_c2.loc[index][c] == 2):
            two_l.append(c.split('_')[0])
            two_df = two_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Train_Targets_c2.loc[index][c] == 3):
            three_l.append(c.split('_')[0])
            three_df = three_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Train_Targets_c2.loc[index][c] == 4):
            four_l.append(c.split('_')[0])
            four_df = four_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
   
    one_df.sort_values(['r_time'], inplace=True)
    two_df.sort_values(['r_time'], inplace=True)
    three_df.sort_values(['r_time'], inplace=True)
    four_df.sort_values(['r_time'], inplace=True)
    #print(one_df.head()) 
    r=1
    for i, r_t in one_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in two_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in three_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in four_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    

In [282]:
reasoner_ranks_test = Test_Targets_c2.copy(deep=True)

for index, row in Test_Targets_c2.iterrows():
    one_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    two_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    three_df = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    four_df  = pd.DataFrame(columns = ['r_name','r_time','r_rank'])

    
    for c in Test_Targets_c2.columns:
        #print(Test_Targets_c2.loc[index][c])
        r = c.split('_')[0]
        #print(Test_Targets_c2.loc[index])
 #       print(c,Test_Targets_c2.loc[index][c])
        if(Test_Targets_c2.loc[index][c] == 1):
            one_l.append(c.split('_')[0])
            one_df = one_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Test_Targets_c2.loc[index][c] == 2):
            two_l.append(c.split('_')[0])
            two_df = two_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Test_Targets_c2.loc[index][c] == 3):
            three_l.append(c.split('_')[0])
            three_df = three_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Test_Targets_c2.loc[index][c] == 4):
            four_l.append(c.split('_')[0])
            four_df = four_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
   
    one_df.sort_values(['r_time'], inplace=True)
    two_df.sort_values(['r_time'], inplace=True)
    three_df.sort_values(['r_time'], inplace=True)
    four_df.sort_values(['r_time'], inplace=True)
    #print(one_df.head()) 
    r=1
    for i, r_t in one_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in two_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in three_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in four_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    

In [284]:
reasoner_ranks_train.tail()

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
1959,5,10,8,9,1,2,3,7,6,4
1960,5,10,7,8,1,2,3,6,9,4
1961,4,10,6,8,1,2,3,5,9,7
1963,5,10,8,9,2,1,3,7,6,4
1964,8,10,4,9,2,3,6,1,5,7


In [285]:
reasoner_ranks_test.tail()           

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
1949,4,10,3,5,8,1,6,2,7,9
1954,2,10,5,6,9,1,4,3,7,8
1955,9,1,4,10,6,2,5,3,7,8
1958,10,1,4,9,6,2,5,3,8,7
1962,9,10,5,6,1,2,3,4,8,7


In [258]:
Test_Targets_c2.tail()

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
1949,3,4,1,3,3,1,3,1,3,3
1954,1,4,1,1,1,1,1,1,1,1
1955,2,1,1,2,1,1,1,1,1,1
1958,1,1,1,1,1,1,1,1,1,1
1962,3,4,2,3,1,1,1,2,3,3


### Rank Prediction Using MTR

In [292]:
mrgs_rf_rank = MultiOutputRegressor(rgs_rf, n_jobs=-1)

mrgs_rf_rank.fit(Train_features, reasoner_ranks_train)
print(mrgs_rf_rank.score(Test_features, reasoner_ranks_test))


0.716131560696


In [294]:
predicted_ranks_test  = mrgs_rf_rank.predict(Test_features)

In [295]:
import scipy.stats as stats
tau, p_value = stats.kendalltau(predicted_ranks_test, reasoner_ranks_test)
print(tau, p_value)

(0.82645844574052607, 0.0)


In [305]:
corr, p_value = stats.spearmanr(predicted_ranks_test, reasoner_ranks_test)#stats.spearmanr([1,2,3,4,5], [5,6,7,8,7])#
print(p_value)

[[  0.00000000e+000   1.08395269e-001   4.19920732e-011   3.36267127e-002
    2.99179839e-032   4.33242683e-013   2.14700787e-008   1.94678313e-030
    3.62198019e-004   1.31114804e-007   6.31271725e-114   3.45303321e-001
    1.36067514e-011   7.73415051e-003   7.64702583e-026   1.71838711e-009
    1.55023452e-004   9.38622032e-021   2.51992258e-002   1.38898126e-005]
 [  1.08395269e-001   0.00000000e+000   2.47955332e-013   5.04953094e-041
    8.40139104e-004   8.90689029e-036   1.80401247e-018   2.78453217e-002
    4.65058364e-024   4.05006678e-018   3.61058800e-002   2.42458511e-164
    7.02282277e-009   1.59587827e-033   6.06405982e-004   8.73263714e-046
    3.57025920e-017   6.31539685e-001   3.21739649e-017   1.47513052e-012]
 [  4.19920732e-011   2.47955332e-013   0.00000000e+000   1.64461966e-004
    5.80621132e-001   6.76605179e-001   1.36578931e-003   6.11961369e-009
    3.24521418e-001   5.86274287e-001   1.49811267e-007   5.34907166e-014
    7.33867053e-108   1.60342164e-00